In [160]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [162]:
test = pd.read_csv("../input/shai-club/test.csv")
train = pd.read_csv("../input/shai-club/train.csv")
train_len = len(train)
test_len = len(test)
print("train length is ", train_len)
print("test length is ", test_len)

In [163]:
train['price'].isna().any()

In [164]:
sns.distplot(train['price'])

In [165]:
train['price'] = np.log(train['price'])
sns.distplot(train['price'])

In [166]:
y=train['price']

In [167]:
df = pd.concat([train, test],axis = 0)

df.head()

In [168]:
df=df.drop("Id",axis=1)

df.drop("price",axis=1 ,inplace=True)
print(df.shape)

In [169]:
df.tail()

In [170]:
df.reset_index(drop=True, inplace=True)

In [171]:
df.tail()

In [172]:
df.info()

In [173]:

df.describe()

In [174]:
df.isnull().sum()

In [175]:
ca_col =[]
for x in df.dtypes.index:
    if df.dtypes[x] == 'object':
        ca_col.append(x)

ca_col   

In [176]:
n_col =[]
for x in df.dtypes.index:
    if df.dtypes[x] == 'float64':
        n_col.append(x)

n_col   

In [177]:
for col in ca_col:
    print (col)
    print(df[col].value_counts())
    print("\n---------------------------------------\n")

In [178]:
sns.countplot(df['cut'])

In [179]:
sns.countplot(df['color'])

In [180]:
sns.countplot(df['clarity'])

In [181]:
for x in n_col:
    plt.figure()
    sns.distplot(df[x])

In [182]:
# train contain the price column , we need it to see the relation between price and the other columns

corr = train.corr()
plt.figure(figsize = (14,8))
sns.heatmap(corr, annot = True , cmap = "BuPu")

In [183]:
train.drop(['x','y','z'],axis=1,inplace=True)
df.drop(['x','y','z'],axis=1,inplace=True)

In [184]:
corr = train.corr()
plt.figure(figsize = (14,8))
sns.heatmap(corr, annot = True , cmap = "BuPu")

In [185]:
df.head()

In [186]:
df.cut=df.cut.replace(['Fair','Good','Very Good','Premium','Ideal'],[1,2,3,4,5])   

In [187]:
df.clarity=df.clarity.replace(['I1', 'SI2', 'SI1','VS2','VS1', 'VVS2','VVS1','IF'],[1,2,3,4,5,6,7,8]) 

In [188]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(drop='first'), ['color'])], remainder='passthrough')
data = columnTransformer.fit_transform(df)
column_name = columnTransformer.get_feature_names()
data =  pd.DataFrame(data, columns= column_name)
data.head(2)

In [189]:
num=['carat','depth','table']
from sklearn.preprocessing import StandardScaler
sub = data[num]
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(sub), columns=sub.columns)

In [190]:
data =data.drop(num,axis =1)
data1 =  pd.concat([data, X], axis=1)
data1.head(3)

In [191]:
test_len

In [192]:
train = data1[:train_len]
test=data1[train_len:]

In [193]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.25, random_state=42)

In [194]:
import xgboost
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor


In [195]:
model_params = {
    'mlp': {
        'model': MLPRegressor(random_state=1),
        'params' : {
            'alpha': [0.0001,0.0005,0.001,0.005],
            'max_iter': [250,350,500]
        }  
    },
    'random_forest': {
        'model': RandomForestRegressor(),
        'params' : {
            
            'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    
        'n_estimators': [100,190,290]
        }
    },
    'xgboost' : {
        'model': xgboost.XGBRegressor(),
        'params': {
            'gamma': [0.5, 1, 1.5, 2],
            'min_child_weight': [1, 5, 10],
            'eta': [ 0.01,0.05,0.1,0.3],
            'max_depth': [ 4,5,6],
            'n_estimators' : [200, 500, 900]
        }
    }
}

from sklearn.model_selection import GridSearchCV
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(train, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

grid search took a lot of time, i couldn't pursue it

In [196]:
regr = MLPRegressor(random_state=1, max_iter=500).fit(x_train, y_train)
regr.score(x_test, y_test)

In [197]:
pred = regr.predict(test)

In [198]:
pred